In [1]:
import os

repo_dir = os.environ.get("REPO_DIR")
code_dir = os.path.join(repo_dir, "code/")
data_dir = os.path.join(repo_dir, "data/")

os.chdir(code_dir)


import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import pickle
import pandas as pd
import sklearn 
import sys
import pandas as pd
from importlib import reload

import matplotlib

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
import seaborn as sns
import geopandas as gpd

import rasterio

from prediction_utils import flatten_raster
from analysis.NL_feature_creation_and_other_NL_processing.nl_helpers import apply_polygon_mask_and_return_flat_array

## In this file, we want to get population counts associated with ADM polygons

We will use Gridded Population of the World V4 (2015), the same product used elsewhere.

### Read shapefiles

In [ ]:
gpdf = pd.read_pickle(data_dir + "int/GDL_HDI/HDI_ADM1_shapefile_clean.p")
adm2 = gpd.read_file(data_dir + "raw/geoBoundaries/geoBoundariesCGAZ_ADM2.geojson")

### Read population total raster (not the same as pop density)

In [ ]:
pop_tot_path = data_dir + "/raw/GPW_pop/gpw_v4_population_count_rev11_2015_30_sec.tif"
src_tot = rasterio.open(pop_tot_path)

In [ ]:
src_tot = rasterio.open(pop_tot_path)

## Get population counts for ADM2 polygons

In [ ]:
sums = []

for index in adm2.index:
    sums.append(apply_polygon_mask_and_return_flat_array(adm2["geometry"].at[index], raster_file=src_tot).sum())
    

In [ ]:
adm2["total_pop"] = np.round(np.array(sums)).astype(int)
adm2["area_sq_km"] = adm2.to_crs({'init': 'epsg:6933'})["geometry"].area.mean()/1e6

In [ ]:
adm2_pop = adm2.set_index("shapeID")[["total_pop", "area_sq_km"]]
adm2_pop.to_pickle("/shares/maps100/data/undp/int/GPW_pop/pop_count_sums_for_ADM2_polygons.p")

In [ ]:
adm2_pop.to_pickle(data_dir + "int/GPW_pop/pop_count_sums_for_ADM2_polygons.p")

## Get population counts for GDL ADM1 polygons

In [ ]:
sums = []

for index in gpdf.index:
    sums.append(apply_polygon_mask_and_return_flat_array(gpdf["geometry"].at[index], raster_file=src_tot).sum())
    

In [ ]:
gpdf["total_pop"] = np.round(np.array(sums)).astype(int)
gpdf["area_sq_km"] = gpdf.to_crs({'init': 'epsg:6933'})["geometry"].area.mean()/1e6

In [ ]:
gpdf_pop = gpdf[["total_pop", "area_sq_km"]]
gpdf_pop.to_pickle(data_dir +" int/GPW_pop/pop_count_sums_for_GDL_ADM1_polygons.p")

## Get population counts for .1 x.1 degree grid

In [ ]:
arr = src_tot.read(1)

In [ ]:
arr[arr < 0]  = 0

In [ ]:
x,y,vals = flatten_raster(arr, src_tot.transform)

In [ ]:
df = pd.DataFrame({"lon":x,"lat":y,"pop_count":vals})

In [ ]:
df["lat10"] = np.round(np.round(df["lat"] + .05,1) - .05,2)
df["lon10"] = np.round(np.round(df["lon"] - .05,1) + .05,2)

In [ ]:
out_df = df.groupby(["lon10","lat10"]).sum().reset_index().drop(columns = ["lon","lat"])

In [ ]:
out_df.to_pickle(data_dir + "int/GPW_pop/pop_count_sums_for_.1x1_grid.p")